In [1]:
import pandas as pd
import glob
import pickle
import requests
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import json
import numpy as np
from numpy import cov
import seaborn as sns
import csv
#извлекает функцию из другого файла ipynb
#from ipynb.fs.full.Methods import download_ats_peak_load

## 1. Загрузка факта нагрузки и генерации

In [2]:
df = pd.read_excel('INITIAL_DATA/Свод_Почасовая выработка 2019 и 2020 обновл.xls',sheet_name=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19], usecols="A:E", skiprows=1,nrows=750, header = None, names = ['timestamp','Solar_generation','External_grid','Total_consumption', 'PVSyst' ])

In [3]:
cdf = pd.concat(df,axis=0, join="inner", ignore_index=True, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

In [4]:
cdf=cdf.dropna(subset=['timestamp', 'External_grid'])
cdf = pd.DataFrame(cdf) 
cdf.set_index('timestamp', inplace=True) # set column 'date' to index

In [5]:
cdf.head(10)

,Solar_generation,External_grid,Total_consumption,PVSyst
timestamp,,,,
2019-06-01 00:00:00,0.000000,271.349396,271.349396,0.00000
2019-06-01 01:00:00,0.000000,243.497849,243.497849,0.00000
2019-06-01 02:00:00,0.000000,258.530853,258.530853,0.00000
2019-06-01 03:00:00,0.000000,255.911179,255.911179,0.00000
2019-06-01 04:00:00,0.000000,257.222229,257.222229,0.00000
2019-06-01 05:00:00,0.275841,260.294891,260.570709,0.00000
2019-06-01 06:00:00,14.505750,233.185471,247.691208,11.81611
2019-06-01 07:00:00,50.503605,193.508102,244.011703,54.20180
2019-06-01 08:00:00,100.483910,198.510956,298.994873,124.11890


In [6]:
cdf.describe()

,Solar_generation,External_grid,Total_consumption,PVSyst
count,13143.000000,13143.000000,13136.000000,13142.000000
mean,31.876580,156.525587,188.417357,69.952133
std,66.501208,142.678545,172.222914,100.775985
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,56.182621,58.468653,0.000000
50%,1.463306,85.453865,107.195679,0.000000
75%,26.580052,265.544342,322.942841,124.118900
max,356.581116,940.887451,1075.956665,405.421449


## 2. Загрузка ЧПН

In [7]:
# specifying the path to csv files
path = "INITIAL_DATA/Peak_hours_fact/"
# xlsx files in the path
file_list = glob.glob(path + "/*.xls")
print(file_list)

['INITIAL_DATA/Peak_hours_fact\\201906_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\201907_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\201908_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\201909_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\201910_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\201911_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\201912_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\202001_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\202002_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\202003_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\202004_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\202005_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\202006_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\202007_KUBANESK_03_calcfacthour.xls', 'INITIAL_DATA/Peak_hours_fact\\20

In [8]:
excl_list = []
for file in file_list:
    df = pd.read_excel(file, usecols="A:B", skiprows=7,nrows=25, names = ['timestamp','Peak_hour'], parse_dates=True)
    df = pd.DataFrame(df) #преобразование в датафрейм
    df["timestamp"]=pd.to_datetime(df["timestamp"],format='%d.%m.%Y').dt.date
    df["Peak_hour"]=pd.to_datetime(df["Peak_hour"], format='%H').dt.time
    df["Peak_hour"]=pd.to_datetime(df["timestamp"].astype(str) + ' ' + df["Peak_hour"].astype(str))
    df_changed=df #создание нового датафрейма
    df_changed.drop(['timestamp'], inplace=True, axis=1) #убрать колонки Date and Time
    df_changed["Peak_hours"]=1
    excl_list.append(df_changed)#добавляем новый элементв в конец
excl_merged = pd.concat(excl_list, axis=0, ignore_index=False) #конкатенация всех листов
excl_merged.columns = ['timestamp','Peak_hour']
excl_merged.set_index('timestamp', inplace=True) # set column 'date' to index
#excl_merged.sort_values(by='timestamp', inplace=True) #сортировка по timeseries в порядке возрастания
#excl_merged.set_index('timestamp', inplace=True) # set column 'date' to index
excl_merged

,Peak_hour
timestamp,
2019-06-03 16:00:00,1
2019-06-04 15:00:00,1
2019-06-05 15:00:00,1
2019-06-06 16:00:00,1
2019-06-07 14:00:00,1
...,...
2021-03-25 11:00:00,1
2021-03-26 11:00:00,1
2021-03-29 20:00:00,1


In [9]:
dt=pd.merge(cdf,excl_merged,how = 'left', on='timestamp')
dt['Peak_hour'] = dt['Peak_hour'].fillna(0)
dt['year'] = dt.index.year
dt['month'] = dt.index.month
dt['day'] =  dt.index.day
dt['hour'] =  dt.index.hour
dt.tail(1000)

,Solar_generation,External_grid,Total_consumption,PVSyst,Peak_hour,year,month,day,hour
timestamp,,,,,,,,,
2020-10-18 15:00:00,35.302059,0.055297,30.820734,166.9904,0.0,2020,10,18,15
2020-10-18 16:00:00,28.380056,6.718463,35.097763,114.3230,0.0,2020,10,18,16
2020-10-18 17:00:00,6.089711,37.089741,43.179451,48.0219,0.0,2020,10,18,17
2020-10-18 18:00:00,0.812735,53.553646,54.366379,0.0000,0.0,2020,10,18,18
2020-10-18 19:00:00,0.000000,55.458897,55.458897,0.0000,0.0,2020,10,18,19
...,...,...,...,...,...,...,...,...,...
2020-12-30 20:00:00,0.000000,72.031883,72.031883,0.0000,1.0,2020,12,30,20
2020-12-30 21:00:00,0.000000,71.898674,71.898674,0.0000,0.0,2020,12,30,21
2020-12-30 22:00:00,0.000000,71.619385,71.619385,0.0000,0.0,2020,12,30,22


## 3. Загрузка ПЧПН

In [10]:
#июнь 2019
dt.loc[(dt['year'] == 2019) & (dt['month'] == 6) & (dt['hour'] >= 8) & (dt['hour'] <= 16), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2019) & (dt['month'] == 6) & (dt['hour'] >= 20) & (dt['hour'] <= 21), 'planned_hour'] = 1 
#июль 2019
dt.loc[(dt['year'] == 2019) & (dt['month'] == 7) & (dt['hour'] >= 8) & (dt['hour'] <= 17), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2019) & (dt['month'] == 7) & (dt['hour'] >= 20) & (dt['hour'] <= 21), 'planned_hour'] = 1
#август 2019
dt.loc[(dt['year'] == 2019) & (dt['month'] == 8) & (dt['hour'] >= 8) & (dt['hour'] <= 21), 'planned_hour'] = 1
#сентябрь 2019
dt.loc[(dt['year'] == 2019) & (dt['month'] == 9) & (dt['hour'] >= 8) & (dt['hour'] <= 15), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2019) & (dt['month'] == 9) & (dt['hour'] >= 18) & (dt['hour'] <= 21), 'planned_hour'] = 1
#октябрь 2019
dt.loc[(dt['year'] == 2019) & (dt['month'] == 10) & (dt['hour'] >= 8) & (dt['hour'] <= 21), 'planned_hour'] = 1
#ноябрь 2019
dt.loc[(dt['year'] == 2019) & (dt['month'] == 11) & (dt['hour'] >= 8) & (dt['hour'] <= 11), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2019) & (dt['month'] == 11) & (dt['hour'] >= 16) & (dt['hour'] <= 21), 'planned_hour'] = 1
#декабрь 2019
dt.loc[(dt['year'] == 2019) & (dt['month'] == 12) & (dt['hour'] >= 8) & (dt['hour'] <= 12), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2019) & (dt['month'] == 12) & (dt['hour'] >= 15) & (dt['hour'] <= 21), 'planned_hour'] = 1
#январь 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 1) & (dt['hour'] >= 8) & (dt['hour'] <= 21), 'planned_hour'] = 1
#февраль 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 2) & (dt['hour'] >= 8) & (dt['hour'] <= 13), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2020) & (dt['month'] == 2) & (dt['hour'] >= 17) & (dt['hour'] <= 21), 'planned_hour'] = 1
#март 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 3) & (dt['hour'] >= 8) & (dt['hour'] <= 21), 'planned_hour'] = 1
#апрель 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 4) & (dt['hour'] >= 8) & (dt['hour'] <= 15), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2020) & (dt['month'] == 4) & (dt['hour'] >= 18) & (dt['hour'] <= 21), 'planned_hour'] = 1
#май 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 5) & (dt['hour'] >= 8) & (dt['hour'] <= 15), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2020) & (dt['month'] == 5) & (dt['hour'] >= 20) & (dt['hour'] <= 21), 'planned_hour'] = 1
#июнь 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 6) & (dt['hour'] >= 8) & (dt['hour'] <= 16), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2020) & (dt['month'] == 6) & (dt['hour'] >= 20) & (dt['hour'] <= 21), 'planned_hour'] = 1
#июль 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 7) & (dt['hour'] >= 8) & (dt['hour'] <= 17), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2020) & (dt['month'] == 7) & (dt['hour'] >= 20) & (dt['hour'] <= 21), 'planned_hour'] = 1
#август 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 8) & (dt['hour'] >= 8) & (dt['hour'] <= 21), 'planned_hour'] = 1
#сентябрь 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 9) & (dt['hour'] >= 8) & (dt['hour'] <= 15), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2020) & (dt['month'] == 9) & (dt['hour'] >= 18) & (dt['hour'] <= 21), 'planned_hour'] = 1
#октябрь 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 10) & (dt['hour'] >= 8) & (dt['hour'] <= 21), 'planned_hour'] = 1
#ноябрь 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 11) & (dt['hour'] >= 8) & (dt['hour'] <= 11), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2020) & (dt['month'] == 11) & (dt['hour'] >= 16) & (dt['hour'] <= 21), 'planned_hour'] = 1
#декабрь 2020
dt.loc[(dt['year'] == 2020) & (dt['month'] == 12) & (dt['hour'] >= 8) & (dt['hour'] <= 12), 'planned_hour'] = 1
dt.loc[(dt['year'] == 2020) & (dt['month'] == 12) & (dt['hour'] >= 15) & (dt['hour'] <= 21), 'planned_hour'] = 1

dt['planned_hour'] = dt['planned_hour'].fillna(0)

In [ ]:
#январь 2022
df.loc[(df['year'] == 2022) & (df['month'] == 1) & (df['hour'] >= 8) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#февраль 2022
df.loc[(df['year'] == 2022) & (df['month'] == 2) & (df['hour'] >= 8) & (df['hour'] <= 13) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2022) & (df['month'] == 2) & (df['hour'] >= 17) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#март 2022
df.loc[(df['year'] == 2022) & (df['month'] == 3) & (df['hour'] >= 8) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#апрель 2022
df.loc[(df['year'] == 2022) & (df['month'] == 4) & (df['hour'] >= 8) & (df['hour'] <= 15) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2022) & (df['month'] == 4) & (df['hour'] >= 18) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#май 2022
df.loc[(df['year'] == 2022) & (df['month'] == 5) & (df['hour'] >= 8) & (df['hour'] <= 15) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2022) & (df['month'] == 5) & (df['hour'] >= 20) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#июнь 2022
df.loc[(df['year'] == 2022) & (df['month'] == 6) & (df['hour'] >= 8) & (df['hour'] <= 16) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2022) & (df['month'] == 6) & (df['hour'] >= 20) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#июль 2022
df.loc[(df['year'] == 2022) & (df['month'] == 7) & (df['hour'] >= 8) & (df['hour'] <= 17) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2022) & (df['month'] == 7) & (df['hour'] >= 20) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#август 2022
df.loc[(df['year'] == 2022) & (df['month'] == 8) & (df['hour'] >= 8) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#сентябрь 2022
df.loc[(df['year'] == 2022) & (df['month'] == 9) & (df['hour'] >= 8) & (df['hour'] <= 15) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2022) & (df['month'] == 9) & (df['hour'] >= 18) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#октябрь 2022
df.loc[(df['year'] == 2022) & (df['month'] == 10) & (df['hour'] >= 8) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#ноябрь 2022
df.loc[(df['year'] == 2022) & (df['month'] == 11) & (df['hour'] >= 8) & (df['hour'] <= 11) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2022) & (df['month'] == 11) & (df['hour'] >= 16) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#декабрь 2022
df.loc[(df['year'] == 2022) & (df['month'] == 12) & (df['hour'] >= 8) & (df['hour'] <= 12) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2022) & (df['month'] == 12) & (df['hour'] >= 15) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1

In [7]:
df = pd.DataFrame(
        {'timestamp': pd.date_range('2023-01-01', '2024-01-01', freq='1H', closed='left')}
     )
#df['Peak_hour'] = 0
df = df.set_index('timestamp')
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] =  df.index.day
df['hour'] =  df.index.hour
df['weekday'] = df.index.weekday
print(df)

                     year  month  day  hour  weekday
timestamp                                           
2023-01-01 00:00:00  2023      1    1     0        6
2023-01-01 01:00:00  2023      1    1     1        6
2023-01-01 02:00:00  2023      1    1     2        6
2023-01-01 03:00:00  2023      1    1     3        6
2023-01-01 04:00:00  2023      1    1     4        6
...                   ...    ...  ...   ...      ...
2023-12-31 19:00:00  2023     12   31    19        6
2023-12-31 20:00:00  2023     12   31    20        6
2023-12-31 21:00:00  2023     12   31    21        6
2023-12-31 22:00:00  2023     12   31    22        6
2023-12-31 23:00:00  2023     12   31    23        6

[8760 rows x 5 columns]


In [8]:
#январь 2023
df.loc[(df['year'] == 2023) & (df['month'] == 1) & (df['hour'] >= 8) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#февраль 2023
df.loc[(df['year'] == 2023) & (df['month'] == 2) & (df['hour'] >= 8) & (df['hour'] <= 13) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2023) & (df['month'] == 2) & (df['hour'] >= 17) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#март 2023
df.loc[(df['year'] == 2023) & (df['month'] == 3) & (df['hour'] >= 8) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#апрель 2023
df.loc[(df['year'] == 2023) & (df['month'] == 4) & (df['hour'] >= 8) & (df['hour'] <= 15) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2023) & (df['month'] == 4) & (df['hour'] >= 18) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#май 2023
df.loc[(df['year'] == 2023) & (df['month'] == 5) & (df['hour'] >= 8) & (df['hour'] <= 15) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2023) & (df['month'] == 5) & (df['hour'] >= 20) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#июнь 2023
df.loc[(df['year'] == 2023) & (df['month'] == 6) & (df['hour'] >= 8) & (df['hour'] <= 16) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2023) & (df['month'] == 6) & (df['hour'] >= 20) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#июль 2023
df.loc[(df['year'] == 2023) & (df['month'] == 7) & (df['hour'] >= 8) & (df['hour'] <= 17) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2023) & (df['month'] == 7) & (df['hour'] >= 20) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#август 2023
df.loc[(df['year'] == 2023) & (df['month'] == 8) & (df['hour'] >= 8) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#сентябрь 2023
df.loc[(df['year'] == 2023) & (df['month'] == 9) & (df['hour'] >= 8) & (df['hour'] <= 15) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2023) & (df['month'] == 9) & (df['hour'] >= 18) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#октябрь 2023
df.loc[(df['year'] == 2023) & (df['month'] == 10) & (df['hour'] >= 8) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#ноябрь 2023
df.loc[(df['year'] == 2023) & (df['month'] == 11) & (df['hour'] >= 8) & (df['hour'] <= 11) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2023) & (df['month'] == 11) & (df['hour'] >= 16) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1
#декабрь 2023
df.loc[(df['year'] == 2023) & (df['month'] == 12) & (df['hour'] >= 8) & (df['hour'] <= 12) & (df['weekday'] <= 4), 'planned_hour'] = 1
df.loc[(df['year'] == 2023) & (df['month'] == 12) & (df['hour'] >= 15) & (df['hour'] <= 21) & (df['weekday'] <= 4), 'planned_hour'] = 1

In [13]:
#экспорт данных по ПЧПН
df_shift=df['planned_hour'].shift(-1).fillna(0).reset_index()
#print(dt_shift)
df_shift.head(900)



,timestamp,planned_hour
0,2023-01-01 00:00:00,0.0
1,2023-01-01 01:00:00,0.0
2,2023-01-01 02:00:00,0.0
3,2023-01-01 03:00:00,0.0
4,2023-01-01 04:00:00,0.0
...,...,...
895,2023-02-07 07:00:00,1.0
896,2023-02-07 08:00:00,1.0
897,2023-02-07 09:00:00,1.0
898,2023-02-07 10:00:00,1.0


In [14]:
df_shift.to_excel('EXPORT_DATA/Planned_hours_2023.xlsx', header = ['Дата','Признак ПЧПН (о.е.)'], index = False) 

In [13]:
#Экспорт данных по фактическим ЧПН

## 4.Загрузка цен

In [14]:
# specifying the path to csv files
path = "INITIAL_DATA/Tariff/"
# xlsx files in the path
file_list = glob.glob(path + "/*.xlsx")
print(file_list)

['INITIAL_DATA/Tariff\\Raskrytie_informatsii_01_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_01_2022_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_02_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_02_2022_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_03_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_03_2022_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_04_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_04_2022_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_05_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_06_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_07_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_08_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_09_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_10_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Raskrytie_informatsii_11_2021_3_TSK.xlsx', 'INITIAL_DATA/Tariff\\Ra

In [15]:
start_date='2021-01-01'
end_date='2021-01-31 23:00:00'
    #парсим январь 2021
tariff = pd.read_excel('INITIAL_DATA/Tariff/Raskrytie_informatsii_01_2021_3_TSK.xlsx', usecols="B:Y", skiprows=250,nrows=31)
tariff = pd.DataFrame(tariff)
i=0
tariff_list=[]
for i in range(len(tariff.index)):
    tariff_changed=tariff.loc[i]
    tariff_list.append(tariff_changed)
tariff_merged = pd.concat(tariff_list, axis=0, ignore_index=False) #конкатенация всех листов
tariff_merged = pd.DataFrame(tariff_merged).reset_index(drop=True)
tariff_merged['timestamp']=pd.date_range(start=start_date, end=end_date, freq='H')
#tariff_merged.set_index('timestamp', inplace=True)
tariff_merged.columns = ['Tariff','timestamp']
tariff_merged=tariff_merged[['timestamp','Tariff']]
print(tariff_merged)
tariff_merged.to_excel('EXPORT_DATA/Tariff.xlsx', sheet_name="3.Стоимость ЭЭ", header = ['Дата','Ставка на покупку электрической энергии, руб/МВт*ч'], index = False) 

              timestamp    Tariff
0   2021-01-01 00:00:00  6590.762
1   2021-01-01 01:00:00  6323.692
2   2021-01-01 02:00:00  6361.602
3   2021-01-01 03:00:00  6280.162
4   2021-01-01 04:00:00  6202.772
..                  ...       ...
739 2021-01-31 19:00:00  6846.472
740 2021-01-31 20:00:00  6837.832
741 2021-01-31 21:00:00  6823.472
742 2021-01-31 22:00:00  6755.662
743 2021-01-31 23:00:00  6324.872

[744 rows x 2 columns]


In [41]:
def tariff_parsing(start_date, end_date, skiprows, Power_cost):
    '''Предназначен для формирования файла *xlsx для загрузки в AMIGO'''
    '''Исходные данные - таблицы тарифов кубаньэнерго, часы пика с атс'''
        #парсим январь 2021
    #подсчитываем количество строк
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    delta = end_date - start_date
    nrows=delta.days+1  
    YEAR=str(start_date.year)
    MONTH=str(start_date.month).zfill(2)
    tariff = pd.read_excel('INITIAL_DATA/Tariff/Raskrytie_informatsii_'+MONTH+'_'+YEAR+'_3_TSK.xlsx', usecols="B:Y", skiprows=skiprows,nrows=nrows)
    tariff = pd.DataFrame(tariff)
    i=0
    tariff_list=[]
    for i in range(len(tariff.index)):
        tariff_changed=tariff.loc[i]
        tariff_list.append(tariff_changed)
    tariff_merged = pd.concat(tariff_list, axis=0, ignore_index=False) #конкатенация всех листов
    tariff_merged = pd.DataFrame(tariff_merged).reset_index(drop=True)
    tariff_merged['timestamp']=pd.date_range(start=start_date, end=end_date, freq='H')
    #tariff_merged.set_index('timestamp', inplace=True)
    tariff_merged.columns = ['Tariff','timestamp']
    tariff_merged=tariff_merged[['timestamp','Tariff']]
    print(tariff_merged)
    
    #парсинг фактических ЧПН
    peak_hours_list=[]
    peak_hours_form=[]
    peak_hours = pd.read_excel('INITIAL_DATA/Peak_hours_fact/'+YEAR+MONTH+'_KUBANESK_03_calcfacthour.xls', usecols="A:B", skiprows=7,nrows=25, names = ['timestamp','Peak_hour'], parse_dates=True)
    peak_hours = pd.DataFrame(peak_hours) #преобразование в датафрейм
    peak_hours["timestamp"]=pd.to_datetime(peak_hours["timestamp"],format='%d.%m.%Y').dt.strftime('%Y-%m-%d %H:%M:%S')
    print(peak_hours)
    
    #Парсинг тарифа на мощность
    Power_cost_table = pd.DataFrame([Power_cost], [pd.to_datetime(start_date).strftime('%Y-%m-%d %H:%M:%S')]).reset_index()
    Power_cost_table.columns=["Date", "Power_cost"]
    print(Power_cost_table)
    #сохранение в файл    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('EXPORT_DATA/Tariff'+'_'+YEAR+'_'+MONTH+'.xlsx', engine='xlsxwriter')

    # Write each dataframe to a different worksheet.
    peak_hours.to_excel(writer, sheet_name='1.Факт_ЧПН', header = ['Дата','Фактический ЧПН'], index = False)
    Power_cost_table.to_excel(writer, sheet_name='2.Ставка мощности',  header = ['Дата','Стоимость мощности (₽/МВт)'], index = False)
    tariff_merged.to_excel(writer, sheet_name='3.Стоимость ЭЭ',  header = ['Дата','Ставка на покупку электрической энергии, руб/МВт*ч'], index = False)
    
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

In [48]:
tariff_parsing('2022-11-01', '2022-11-30 23:00:00', 243, 893803.03)

              timestamp    Tariff
0   2022-11-01 00:00:00  6136.916
1   2022-11-01 01:00:00  6127.556
2   2022-11-01 02:00:00  6093.766
3   2022-11-01 03:00:00  5921.346
4   2022-11-01 04:00:00  6117.766
..                  ...       ...
715 2022-11-30 19:00:00  7774.706
716 2022-11-30 20:00:00  7685.426
717 2022-11-30 21:00:00  7653.636
718 2022-11-30 22:00:00  7504.906
719 2022-11-30 23:00:00  6801.936

[720 rows x 2 columns]
              timestamp  Peak_hour
0   2022-11-01 00:00:00         19
1   2022-11-02 00:00:00         19
2   2022-11-03 00:00:00         19
3   2022-11-07 00:00:00         19
4   2022-11-08 00:00:00         19
5   2022-11-09 00:00:00         19
6   2022-11-10 00:00:00         19
7   2022-11-11 00:00:00         19
8   2022-11-14 00:00:00         19
9   2022-11-15 00:00:00         19
10  2022-11-16 00:00:00         19
11  2022-11-17 00:00:00         19
12  2022-11-18 00:00:00         19
13  2022-11-21 00:00:00         19
14  2022-11-22 00:00:00         20
15  2022

In [45]:
#Февраль 2021
tariff_parsing('2021-02-01', '2021-02-28 23:00:00', 229, 812867.45)

              timestamp    Tariff
0   2021-02-01 00:00:00  6285.472
1   2021-02-01 01:00:00  6260.762
2   2021-02-01 02:00:00  6216.192
3   2021-02-01 03:00:00  6162.182
4   2021-02-01 04:00:00  6221.972
..                  ...       ...
667 2021-02-28 19:00:00  6907.062
668 2021-02-28 20:00:00  6898.462
669 2021-02-28 21:00:00  6886.152
670 2021-02-28 22:00:00  6831.632
671 2021-02-28 23:00:00  6731.822

[672 rows x 2 columns]
              timestamp  Peak_hour
0   2021-02-01 00:00:00         19
1   2021-02-02 00:00:00         20
2   2021-02-03 00:00:00         19
3   2021-02-04 00:00:00         20
4   2021-02-05 00:00:00         19
5   2021-02-08 00:00:00         19
6   2021-02-09 00:00:00         19
7   2021-02-10 00:00:00         20
8   2021-02-11 00:00:00         20
9   2021-02-12 00:00:00         20
10  2021-02-15 00:00:00         19
11  2021-02-16 00:00:00         19
12  2021-02-17 00:00:00         20
13  2021-02-18 00:00:00         20
14  2021-02-19 00:00:00         11
15  2021

In [22]:
#Июнь 2023
tariff_parsing('2023-06-01', '2023-06-30 23:00:00', 243, 927250.16)

              timestamp    Tariff
0   2022-06-01 00:00:00  6726.422
1   2022-06-01 01:00:00  6588.292
2   2022-06-01 02:00:00  6518.662
3   2022-06-01 03:00:00  6472.702
4   2022-06-01 04:00:00  6388.872
..                  ...       ...
715 2022-06-30 19:00:00  7129.222
716 2022-06-30 20:00:00  7072.642
717 2022-06-30 21:00:00  7139.272
718 2022-06-30 22:00:00  6805.092
719 2022-06-30 23:00:00  6468.842

[720 rows x 2 columns]
     timestamp  Peak_hour
0   2022-06-01         15
1   2022-06-02         16
2   2022-06-03         15
3   2022-06-06         14
4   2022-06-07         16
5   2022-06-08         15
6   2022-06-09         15
7   2022-06-10         15
8   2022-06-14         14
9   2022-06-15         15
10  2022-06-16         15
11  2022-06-17         15
12  2022-06-20         16
13  2022-06-21         15
14  2022-06-22         15
15  2022-06-23         14
16  2022-06-24         12
17  2022-06-27         12
18  2022-06-28         15
19  2022-06-29         15
20  2022-06-30        

In [161]:
def tariff_parsing_mosenergo(start_date, end_date):
    '''Предназначен для формирования файла *xlsx для загрузки в AMIGO'''
    '''Исходные данные - таблицы мосэнерго, часы пика с атс, данные принимаются с time2save'''
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    delta = end_date - start_date
    nrows=(delta.days+1)*5
    skiprows=10+(5*(delta.days+1)+2+1)*3+1
    YEAR=str(start_date.year)
    MONTH=str(start_date.month).zfill(2)
    tariff = pd.read_excel('INITIAL_DATA/Tariff/Mosenergosbit_'+MONTH+'_'+YEAR+'_4_TSK_670_10000.xlsx', sheet_name='4 ЦК', usecols="B:Y", skiprows=skiprows,nrows=nrows)
    tariff = pd.DataFrame(tariff)
    i=0
    tariff_list=[]
    for i in range(delta.days+1):
        tariff_changed=tariff.loc[i*5]
        tariff_list.append(tariff_changed)
    tariff_merged = pd.concat(tariff_list, axis=0, ignore_index=False) #конкатенация всех листов
    tariff_merged = pd.DataFrame(tariff_merged).reset_index(drop=True)
    tariff_merged['timestamp']=pd.date_range(start=start_date, end=end_date, freq='H')
    #tariff_merged.set_index('timestamp', inplace=True)
    tariff_merged.columns = ['Tariff','timestamp']
    tariff_merged=tariff_merged[['timestamp','Tariff']]
    print(tariff_merged)
    
    #парсинг фактических ЧПН
    peak_hours_list=[]
    peak_hours_form=[]
    peak_hours = pd.read_excel('INITIAL_DATA/Peak_hours_fact/'+YEAR+MONTH+'_MOSENERG_46_calcfacthour.xls', usecols="A:B", skiprows=7,nrows=25, names = ['timestamp','Peak_hour'], parse_dates=True)
    peak_hours = pd.DataFrame(peak_hours) #преобразование в датафрейм
    peak_hours["timestamp"]=pd.to_datetime(peak_hours["timestamp"],format='%d.%m.%Y').dt.date
    print(peak_hours)
    
    #Парсинг тарифа на мощность
    #Power_cost_table = pd.DataFrame([Power_cost], [start_date]).reset_index()
    #Power_cost_table.columns=["Date", "Power_cost"]
    #print(Power_cost_table)
    
    #Парсинг тарифа на мощность
    Power_cost_table_skiprows=10+(5*(delta.days+1)+2+1)*4+(4*(delta.days+1)+2+1)+5
    Power_cost_table = pd.read_excel('INITIAL_DATA/Tariff/Mosenergosbit_'+MONTH+'_'+YEAR+'_4_TSK_670_10000.xlsx', sheet_name='4 ЦК', usecols="M:M", skiprows=Power_cost_table_skiprows,nrows=1)
    Power_cost_table = pd.DataFrame(Power_cost_table)
    Power_cost_table = pd.DataFrame([Power_cost_table.iloc[0,0]], [start_date]).reset_index()
    Power_cost_table.columns=["Date", "Power_cost"]
    print(Power_cost_table)
    
    #Парсинг тарифа на передачу
    Power_transmission_cost_table_skiprows=10+(5*(delta.days+1)+2+1)*4+(4*(delta.days+1)+2+1)+14
    Power_transmission_cost_table = pd.read_excel('INITIAL_DATA/Tariff/Mosenergosbit_'+MONTH+'_'+YEAR+'_4_TSK_670_10000.xlsx', sheet_name='4 ЦК', usecols="L:L", skiprows=Power_transmission_cost_table_skiprows,nrows=1)
    Power_transmission_cost_table = pd.DataFrame([Power_transmission_cost_table.iloc[0,0]], [start_date]).reset_index()
    Power_transmission_cost_table.columns=["Date", "Power_transmission_cost"]
    print(Power_transmission_cost_table)
    
    #сохранение в файл    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('EXPORT_DATA/Tariff_MOSENERG'+'_'+YEAR+'_'+MONTH+'.xlsx', engine='xlsxwriter')

    # Write each dataframe to a different worksheet.
    peak_hours.to_excel(writer, sheet_name='1.Факт_ЧПН', header = ['Дата','Фактический ЧПН'], index = False)
    Power_cost_table.to_excel(writer, sheet_name='2.Ставка мощности',  header = ['Дата','Стоимость мощности (₽/МВт)'], index = False)
    tariff_merged.to_excel(writer, sheet_name='3.Стоимость ЭЭ',  header = ['Дата','Ставка на покупку электрической энергии, руб/МВт*ч'], index = False)
    Power_transmission_cost_table.to_excel(writer, sheet_name='4.Ставка мощности на передачу',  header = ['Дата','Стоимость передачи мощности (₽/МВт)'], index = False)
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

In [175]:
#Январь 2023
tariff_parsing_mosenergo('2021-12-01', '2021-12-31 23:00:00')

              timestamp   Tariff
0   2021-12-01 00:00:00  1597.85
1   2021-12-01 01:00:00  1526.34
2   2021-12-01 02:00:00  1474.33
3   2021-12-01 03:00:00  1458.58
4   2021-12-01 04:00:00  1549.42
..                  ...      ...
739 2021-12-31 19:00:00  2287.76
740 2021-12-31 20:00:00  2300.82
741 2021-12-31 21:00:00  2277.83
742 2021-12-31 22:00:00  2164.10
743 2021-12-31 23:00:00  1992.79

[744 rows x 2 columns]
     timestamp  Peak_hour
0   2021-12-01         18
1   2021-12-02         18
2   2021-12-03         18
3   2021-12-06         17
4   2021-12-07         17
5   2021-12-08         18
6   2021-12-09         18
7   2021-12-10         18
8   2021-12-13         17
9   2021-12-14         18
10  2021-12-15         18
11  2021-12-16         17
12  2021-12-17         18
13  2021-12-20         18
14  2021-12-21         18
15  2021-12-22         18
16  2021-12-23         18
17  2021-12-24         18
18  2021-12-27         18
19  2021-12-28         18
20  2021-12-29         18
21  2021

In [15]:
#Парсер для формирования данных по Кубаньэнерго
def tariff_parsing1(start_date, end_date, skiprows):
        #парсим январь 2021
    #подсчитываем количество строк
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    delta = end_date - start_date
    nrows=delta.days+1  
    YEAR=str(start_date.year)
    MONTH=str(start_date.month).zfill(2)
    tariff = pd.read_excel('INITIAL_DATA/Tariff/Raskrytie_informatsii_'+MONTH+'_'+YEAR+'_3_TSK.xlsx', usecols="B:Y", skiprows=skiprows,nrows=nrows)
    tariff = pd.DataFrame(tariff)
    i=0
    tariff_list=[]
    for i in range(len(tariff.index)):
        tariff_changed=tariff.loc[i]
        tariff_list.append(tariff_changed)
    tariff_merged = pd.concat(tariff_list, axis=0, ignore_index=False) #конкатенация всех листов
    tariff_merged = pd.DataFrame(tariff_merged).reset_index(drop=True)
    tariff_merged['timestamp']=pd.date_range(start=start_date, end=end_date, freq='H')
    #tariff_merged.set_index('timestamp', inplace=True)
    tariff_merged.columns = ['Tariff','timestamp']
    tariff_merged=tariff_merged[['timestamp','Tariff']]
    print(tariff_merged)
    
    #парсинг фактических ЧПН
    peak_hours_list=[]
    peak_hours_form=[]
    peak_hours = pd.read_excel('INITIAL_DATA/Peak_hours_fact/'+YEAR+MONTH+'_KUBANESK_03_calcfacthour.xls', usecols="A:B", skiprows=7,nrows=25, names = ['timestamp','Peak_hour'], parse_dates=True)
    peak_hours = pd.DataFrame(peak_hours) #преобразование в датафрейм
    peak_hours["timestamp"]=pd.to_datetime(peak_hours["timestamp"],format='%d.%m.%Y').dt.date
    peak_hours["Peak_hour"]=pd.to_datetime(peak_hours["Peak_hour"], format='%H').dt.time
    peak_hours["Peak_hour"]=pd.to_datetime(peak_hours["timestamp"].astype(str) + ' ' + peak_hours["Peak_hour"].astype(str))
    peak_hours.drop(['timestamp'], inplace=True, axis=1) #убрать колонки Date and Time
    peak_hours["Peak_hours"]=1
    peak_hours_list.append(peak_hours)#добавляем новый элементв в конец
    peak_hours_merged = pd.concat(peak_hours_list, axis=0, ignore_index=False) #конкатенация всех листов
    peak_hours_merged.columns = ['timestamp','Peak_hour']
    peak_hours_form = pd.DataFrame(pd.date_range(start=start_date, end=end_date, freq='H'), columns=['timestamp'])
    peak_hours_form['timestamp']=peak_hours_form
    peak_hours_form_merged=pd.merge(peak_hours_form,peak_hours_merged,how = 'left', on='timestamp')
    peak_hours_form_merged['Peak_hour'] = peak_hours_form_merged['Peak_hour'].fillna(0)
    print(peak_hours_form_merged)
    
    #Парсинг тарифа на мощность
    
    #сохранение в файл    
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter('EXPORT_DATA/Tariff'+'_'+YEAR+'_'+MONTH+'.xlsx', engine='xlsxwriter')

    # Write each dataframe to a different worksheet.
    peak_hours_form_merged.to_excel(writer, sheet_name='1.Факт_ЧПН', header = ['Дата','Фактический ЧПН'], index = False)
    tariff_merged.to_excel(writer, sheet_name='3.Стоимость ЭЭ',  header = ['Дата','Ставка на покупку электрической энергии, руб/МВт*ч'], index = False)

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

In [19]:
tariff_parsing1('2022-11-01', '2022-11-30 23:00:00', 144)

              timestamp    Tariff
0   2022-11-01 00:00:00  2707.106
1   2022-11-01 01:00:00  2697.746
2   2022-11-01 02:00:00  2663.956
3   2022-11-01 03:00:00  2491.536
4   2022-11-01 04:00:00  2687.956
..                  ...       ...
715 2022-11-30 19:00:00  4344.896
716 2022-11-30 20:00:00  4255.616
717 2022-11-30 21:00:00  4223.826
718 2022-11-30 22:00:00  4075.096
719 2022-11-30 23:00:00  3372.126

[720 rows x 2 columns]
              timestamp  Peak_hour
0   2022-11-01 00:00:00        0.0
1   2022-11-01 01:00:00        0.0
2   2022-11-01 02:00:00        0.0
3   2022-11-01 03:00:00        0.0
4   2022-11-01 04:00:00        0.0
..                  ...        ...
715 2022-11-30 19:00:00        1.0
716 2022-11-30 20:00:00        0.0
717 2022-11-30 21:00:00        0.0
718 2022-11-30 22:00:00        0.0
719 2022-11-30 23:00:00        0.0

[720 rows x 2 columns]
